In [53]:
"""
Just run all of these to completion to preprocess the data and load it into the raw folder

"""

'\nJust run all of these to completion to preprocess the data and load it into the raw folder\n\n'

In [54]:
"""
Create a Spark Session

"""

from pyspark.sql import SparkSession
import pyarrow.parquet as pq
spark = (
    SparkSession.builder.appName("MAST30034 Assignment")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

23/08/17 13:53:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/08/17 13:53:12 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [55]:
"""
Data preprocessing for yellow taxi tripdata 2019/02-07 parquet files 

"""

from pyspark.sql.functions import col, month, hour, to_date, dayofweek

# data cleansing for yellow taxi 
yellow_feb = spark.read.parquet("data/landing/yellow_tripdata_2019-02.parquet")
yellow_march = spark.read.parquet("data/landing/yellow_tripdata_2019-03.parquet")
yellow_april = spark.read.parquet("data/landing/yellow_tripdata_2019-04.parquet")
yellow_may = spark.read.parquet("data/landing/yellow_tripdata_2019-05.parquet")
yellow_june= spark.read.parquet("data/landing/yellow_tripdata_2019-06.parquet")
yellow_july = spark.read.parquet("data/landing/yellow_tripdata_2019-07.parquet")

# merge the yellow taxi data
yellow_taxi_all = yellow_feb.unionAll(yellow_march).unionAll(yellow_april).unionAll(yellow_may).unionAll(yellow_june).unionAll(yellow_july)

# remove irrelevant columns
irr_columns = ['VendorID', 'passenger_count', 'store_and_fwd_flag', 'RatecodeID', 'payment_type', 'extra', 'mta_tax', 
                'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'airport_fee']
yellow_taxi_all = yellow_taxi_all.drop(*irr_columns)

# remove null values for columns that I need to exclude null values 
yellow_drop_na_cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance', 
                'PULocationID', 'DOLocationID', 'fare_amount']

yellow_taxi_all = yellow_taxi_all.dropna(subset=yellow_drop_na_cols)
yellow_taxi_all.show()

# Save the DataFrame to a Parquet file
yellow_taxi_all.repartition(1).write.format("parquet").mode("append").save("data/raw/raw_yellow_taxi")

# NOTE - as the data saves as a folder as opposed to a file, you will have to rename the parquet file in the output folder
# please rename the generated parquet file to raw_yellow_taxi.parquet and place in in the raw folder and also remove the generated folder

+--------------------+---------------------+-------------+------------+------------+-----------+------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|trip_distance|PULocationID|DOLocationID|fare_amount|total_amount|
+--------------------+---------------------+-------------+------------+------------+-----------+------------+
| 2019-02-01 00:59:04|  2019-02-01 01:07:27|          2.1|          48|         234|        9.0|        12.3|
| 2019-02-01 00:33:09|  2019-02-01 01:03:58|          9.8|         230|          93|       32.0|        33.3|
| 2019-02-01 00:09:03|  2019-02-01 00:09:16|          0.0|         145|         145|        2.5|         3.8|
| 2019-02-01 00:45:38|  2019-02-01 00:51:10|          0.8|          95|          95|        5.5|         6.8|
| 2019-02-01 00:25:30|  2019-02-01 00:28:14|          0.8|         140|         263|        5.0|         6.3|
| 2019-02-01 00:38:02|  2019-02-01 00:40:57|          0.8|         229|         141|        4.5|         5.8|
| 2019-02-

In [56]:
"""
External data pre-processing event data: https://data.cityofnewyork.us/Recreation/Parks-Special-Events/6v4b-5gp4/data

"""

import pandas as pd
from datetime import datetime
import pandas as pd
from pyspark.sql.functions import lit
from pathlib import Path  

# event data pre-processing
event = pd.read_csv("data/landing/Parks_Special_Events.csv")

# remove irrelevant columns 
event = event.drop(columns = ["Group Name/Partner", "Unit", "LocationType", "Event Name", "Event Type", "Classification", "Attendance",
                                                "Location", "Category", "Audience"])

# parse one column in two columns (date and time)
date_format = "%m/%d/%Y %I:%M:%S %p"
event["Date and Time"] = pd.to_datetime(event["Date and Time"], format= date_format)
event.rename(columns = { "Date and Time" :"date"}, inplace = True)
event['hour'] = event["date"].dt.hour
event['date'] = event["date"].dt.date

event = event.rename(columns={"date": "date2", "Borough": "Borough2", "hour": "hour2"})

filepath = Path('data/raw/raw_events.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
event.to_csv(filepath)  

In [57]:
"""
External data pre-processing for taxi zone lookup data

"""

import pandas as pd
from datetime import datetime
import pandas as pd
from pathlib import Path  

# event data pre-processing
taxi_zone = pd.read_csv("data/landing/taxi+_zone_lookup.csv").drop(columns=["Zone", "service_zone"])
taxi_zone = taxi_zone.rename(columns={"LocationID": "PULocationID"})

taxi_zone = taxi_zone.query("Borough != 'Unknown'")

print(taxi_zone)

filepath = Path('data/raw/raw_taxi_zones.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
taxi_zone.to_csv(filepath)  


     PULocationID        Borough
0               1            EWR
1               2         Queens
2               3          Bronx
3               4      Manhattan
4               5  Staten Island
..            ...            ...
258           259          Bronx
259           260         Queens
260           261      Manhattan
261           262      Manhattan
262           263      Manhattan

[263 rows x 2 columns]


In [59]:
"""
External data pre-processing for weather data at https://www.kaggle.com/datasets/alejopaullier/new-york-city-weather-data-2019 

"""

import pandas as pd
from datetime import datetime
import pandas as pd
from pathlib import Path  

import pandas as pd 
weather = pd.read_csv("data/landing/nyc_temperature.csv")

# parse data by row (01/02/19 - 31/07/2019)
weather = weather.iloc[31:211 + 1]
weather = weather.reset_index(drop=True)

# checked there's no snow from feb to july 
# delete snow relevant columns 
del_col = ['tmax','tmin','departure','HDD','CDD','snow_depth']
for column in del_col:
    del weather[column]

# check if rained/snowed or not 
weather['rained'] = weather['precipitation'] > '0'
weather['rained'] = weather['rained'].map({True: 'Yes', False: 'No'})

# check if snowed or not
weather['snowed'] = weather['new_snow'] > '0'
weather['snowed'] = weather['snowed'].map({True: 'Yes', False: 'No'})

del_col2 = ['precipitation', 'new_snow']
for column in del_col2:
    del weather[column]

# categorize average temperatures in 3 categories; cold, moderate, hot
temp_categorize = {
    'Cold': (-float('inf'), 50),
    'Moderate': (50, 77),
    'Hot': (77, float('inf'))
}

def categorize_temp(temp):
    for category, (lower, upper) in temp_categorize.items():
        if lower <= temp < upper:
            return category

weather['tavg'] = weather['tavg'].apply(categorize_temp)
weather.rename(columns={'tavg': 'temp_categorized'}, inplace=True)

# convert weather date to timestamp
weather['date'] = pd.to_datetime(weather['date'], format='%d/%m/%y')

print(weather)

filepath = Path('data/raw/raw_weather.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
weather.to_csv(filepath)  

          date temp_categorized rained snowed
0   2019-02-01             Cold     No     No
1   2019-02-02             Cold     No     No
2   2019-02-03             Cold     No     No
3   2019-02-04             Cold     No     No
4   2019-02-05         Moderate     No     No
..         ...              ...    ...    ...
176 2019-07-27              Hot     No     No
177 2019-07-28              Hot    Yes     No
178 2019-07-29              Hot     No     No
179 2019-07-30              Hot    Yes     No
180 2019-07-31              Hot    Yes     No

[181 rows x 4 columns]


In [60]:
spark.stop()